<a href="https://colab.research.google.com/github/NicoloAvesani/Mining_Project/blob/main/that's_good_OK_APRIORI%2C_FP_GROWTH_and_ECLAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset generator
https://github.com/jeffheaton/papers/blob/master/2016/ieee-freq-item/GenerateFreqData.py


In [ ]:
# This Python script was used to collect the data for following paper/conference:
#
# Heaton, J. (2016, April). Comparing Dataset Characteristics that Favor the Apriori, 
# Eclat or FP-Growth Frequent Itemset Mining Algorithms. In SoutheastCon 2015 (pp. 1-6). IEEE.
#
# http://www.jeffheaton.com
#

# Generate benchmark data for frequent itemset mining.
__author__ = 'jheaton'
import random
import csv
from tqdm import tqdm

def sizeof_fmt(num):
    for x in ['','k','m','g']:
        if num < 1000.0:
            return "%3.1f%s" % (num, x)
        num /= 1000.0
    return "%3.1f%s" % (num, 't')

def generate_itemset(row_count, max_per_basket, num_freq_sets, item_count, prob_frequent):
    '''
    Generate a dataset of frequent items. These paramaters can be changed to 
    determine the type of data to generate.

    :param int row_count: The number of rows in the dataset.
    :param int max_per_basket: Maximum number of items per basket.
    :param int num_freq_sets: The number of unique frequent item sets.
    :param int item_count: The number of unique items.
    :param float prob_frequent: The probability of a basket containing a frequent itemset.
    '''
    # Generate the data
    pop_frequent = ["F"+str(n) for n in range(0,max_per_basket)]
    pop_regular = ["I"+str(n) for n in range(max_per_basket,item_count)]
    freq_itemsets = []

    # Create a filename that encodes the max_per_basket and basket_count into
    # the filename.
    filename = str(prob_frequent)+"_tsz" \
        + str(max_per_basket)+'_tct' \
         +sizeof_fmt(row_count)+'.txt'

    for i in tqdm(range(num_freq_sets),desc=f"{filename}:pass 1/2"):
        cnt = random.randint(1,max_per_basket)
        freq_itemsets.append(random.sample(pop_frequent,cnt))

    with open(filename, 'w') as f:
        for i in tqdm(range(row_count),desc=f"{filename}:pass 2/2"):
            line = []

            cnt = random.randint(1,max_per_basket)
            if random.random()<=prob_frequent:
                idx = random.randint(0,len(freq_itemsets)-1)
                for j in range(len(freq_itemsets[idx])):
                    line.append(freq_itemsets[idx][j])

            needed = max(0,cnt - len(line))
            line = line + random.sample(pop_regular,needed)

            f.write(" ".join(line)+"\n")

random.seed(1000)

In [ ]:
dataset = generate_itemset(200000, 50, 100, 500, 0.5)

    #Generate a dataset of frequent items. These paramaters can be changed to 
    #determine the type of data to generate.

    #:param int row_count: The number of rows in the dataset. (in this case they are 10000)
    #:param int max_per_basket: Maximum number of items per basket. (max 100 items per basket)
    #:param int num_freq_sets: The number of unique frequent item sets. (100 freq item sets)
    #:param int item_count: The number of unique items. (500 unique items)
    #:param float prob_frequent: The probability of a basket containing a frequent itemset. (50% is the probability that a basket contains a freq itemset)

0.5_tsz50_tct200.0k.txt:pass 2/2: 100%|██████████| 200000/200000 [00:04<00:00, 47323.26it/s]


In [ ]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('0.5_tsz50_tct200.0k.txt')

In [ ]:
# I want the dataset to be a np.array
dataset_array = np.array(dataset)

In [ ]:
dataset_array[0]

array(['F8 F2 F26 F3 F25 F6 F43 F24 F42 F4 F13 F16 F36 F5 F47 F19 F35 F39 F48 F10 F22 F23 F34 F11 F18 F29 F7 F20 F30 F33 F27 F44 F21'],
      dtype=object)

In [ ]:
# since the array contains baskets with only a string of element and not a list of elements, I need to fix this thing
new_array = []
for i in range(len(dataset_array)):
    split_elements = np.char.split(dataset_array[i].astype(str))
    ok_split_elements = (split_elements[0])
    new_array.append(ok_split_elements)

In [ ]:
# now it works
new_array[0][1]

'F2'

In [ ]:
new_array[0]

['F8',
 'F2',
 'F26',
 'F3',
 'F25',
 'F6',
 'F43',
 'F24',
 'F42',
 'F4',
 'F13',
 'F16',
 'F36',
 'F5',
 'F47',
 'F19',
 'F35',
 'F39',
 'F48',
 'F10',
 'F22',
 'F23',
 'F34',
 'F11',
 'F18',
 'F29',
 'F7',
 'F20',
 'F30',
 'F33',
 'F27',
 'F44',
 'F21']

In [ ]:
len(new_array[0])

33

#apriori



In [ ]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#apriori algorithm using the library--> it shows me all the possible k-frequent itemsets with the support

from apyori import apriori
import time

# Example dataset
dataset = new_array

# Minimum support threshold
min_support = 0.25

start = time.time()
# Apply the Apriori algorithm
results = list(apriori(dataset, min_support=min_support))

# Print all frequent itemsets
for itemset in results:
    print(itemset)

end = time.time()

print('Time required:', end - start)


RelationRecord(items=frozenset({'F11'}), support=0.2664113320566603, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'F11'}), confidence=0.2664113320566603, lift=1.0)])
RelationRecord(items=frozenset({'F12'}), support=0.29085145425727127, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'F12'}), confidence=0.29085145425727127, lift=1.0)])
RelationRecord(items=frozenset({'F13'}), support=0.28638143190715953, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'F13'}), confidence=0.28638143190715953, lift=1.0)])
RelationRecord(items=frozenset({'F14'}), support=0.28571142855714277, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'F14'}), confidence=0.28571142855714277, lift=1.0)])
RelationRecord(items=frozenset({'F15'}), support=0.2703363516817584, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'F15'}), confidence=0.270336351

#fp-growth
https://github.com/JackHCC/Apriori-and-FP_Growth/blob/master/FP_growth.py


In [ ]:
#fp-growth

# fp-growth

from collections import defaultdict, namedtuple
import time

def find_frequent_itemsets(transactions, minimum_support, include_support=False):
    # Function to find frequent itemsets in the transactions dataset

    items = defaultdict(lambda: 0)

    for transaction in transactions:
        for item in transaction:
            items[item] += 1

    # Filtering frequent 1-itemsets
    items = dict((item, support) for item, support in items.items()
        if support >= minimum_support)

    def clean_transaction(transaction):
        transaction = filter(lambda v: v in items, transaction)
        transaction_list = list(transaction)
        transaction_list.sort(key=lambda v: items[v], reverse=True)
        return transaction_list

    # Constructing the FP-tree
    master = FPTree()
    for transaction in map(clean_transaction, transactions):
        master.add(transaction)

    def find_with_suffix(tree, suffix):
        for item, nodes in tree.items():
            support = sum(n.count for n in nodes)
            if support >= minimum_support and item not in suffix:
                found_set = [item] + suffix
                yield (found_set, support) if include_support else found_set

                cond_tree = conditional_tree_from_paths(tree.prefix_paths(item))
                for s in find_with_suffix(cond_tree, found_set):
                    yield s

    for itemset in find_with_suffix(master, []):
        yield itemset

class FPTree(object):
    # FP-tree data structure

    Route = namedtuple('Route', 'head tail')

    def __init__(self):
        # Initialize the root node and routes
        self._root = FPNode(self, None, None)
        self._routes = {}

    @property
    def root(self):
        # Create the root node
        return self._root

    def add(self, transaction):
        # Add a transaction to the tree
        point = self._root

        for item in transaction:
            next_point = point.search(item)
            if next_point:
                # Current node already exists
                next_point.increment()
            else:
                # Create a new node
                next_point = FPNode(self, item)
                point.add(next_point)

                # Update the routes
                self._update_route(next_point)

            point = next_point

    def _update_route(self, point):
        assert self is point.tree

        try:
            route = self._routes[point.item]
            route[1].neighbor = point  # route[1] is the tail
            self._routes[point.item] = self.Route(route[0], point)
        except KeyError:
            # Start a new node
            self._routes[point.item] = self.Route(point, point)

    def items(self):
        # Iterate over the items in the tree
        for item in self._routes.keys():
            yield (item, self.nodes(item))

    def nodes(self, item):
        # Iterate over the nodes associated with an item
        try:
            node = self._routes[item][0]
        except KeyError:
            return

        while node:
            yield node
            node = node.neighbor

    def prefix_paths(self, item):
        # Generate prefix paths for an item
        def collect_path(node):
            path = []
            while node and not node.root:
                path.append(node)
                node = node.parent
            path.reverse()
            return path

        return (collect_path(node) for node in self.nodes(item))

    def inspect(self):
        # Print the tree structure
        self.root.inspect(1)

        for item, nodes in self.items():
            for node in nodes:
                print('    %r' % node)

def conditional_tree_from_paths(paths):
    # Construct a conditional tree from paths
    tree = FPTree()
    condition_item = None
    items = set()

    for path in paths:
        if condition_item is None:
            condition_item = path[-1].item

        point = tree.root
        for node in path:
            next_point = point.search(node.item)
            if not next_point:
                # Add a new node to the tree
                items.add(node.item)
                count = node.count if node.item == condition_item else 0
                next_point = FPNode(tree, node.item, count)
                point.add(next_point)
                tree._update_route(next_point)
            point = next_point

    assert condition_item is not None

    # Calculate the counts for nodes
    for path in tree.prefix_paths(condition_item):
        count = path[-1].count
        for node in reversed(path[:-1]):
            node._count += count

    return tree

class FPNode(object):
    # FP-tree node

    def __init__(self, tree, item, count=1):
        self._tree = tree
        self._item = item
        self._count = count
        self._parent = None
        self._children = {}
        self._neighbor = None

    def add(self, child):
        # Add a child node

        if not isinstance(child, FPNode):
            raise TypeError("Can only add other FPNodes as children")

        if not child.item in self._children:
            self._children[child.item] = child
            child.parent = self

    def search(self, item):
        # Search for a child node with the given item
        try:
            return self._children[item]
        except KeyError:
            return None

    def __contains__(self, item):
        return item in self._children

    @property
    def tree(self):
        return self._tree

    @property
    def item(self):
        return self._item

    @property
    def count(self):
        return self._count

    def increment(self):
        if self._count is None:
            raise ValueError("Root nodes have no associated count.")
        self._count += 1

    @property
    def root(self):
        return self._item is None and self._count is None

    @property
    def leaf(self):
        return len(self._children) == 0

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a parent.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a parent from another tree.")
        self._parent = value

    @property
    def neighbor(self):
        return self._neighbor

    @neighbor.setter
    def neighbor(self, value):
        if value is not None and not isinstance(value, FPNode):
            raise TypeError("A node must have an FPNode as a neighbor.")
        if value and value.tree is not self.tree:
            raise ValueError("Cannot have a neighbor from another tree.")
        self._neighbor = value

    @property
    def children(self):
        return tuple(self._children.values())

    def inspect(self, depth=0):
        for child in self.children:
            child.inspect(depth + 1)

    def __repr__(self):
        if self.root:
            return "<%s (root)>" % type(self).__name__
        return "<%s %r (%r)>" % (type(self).__name__, self.item, self.count)


In [ ]:
# Dataset
dataset = new_array
# Main function
if __name__ == '__main__':
    # Call find_frequent_itemsets() to generate frequent itemsets
    start = time.time()

    total_baskets = len(dataset)
    min_support = 0.25 * total_baskets

    frequent_itemsets = find_frequent_itemsets(dataset, minimum_support=min_support, include_support=True)

    result = []
    for itemset, support in frequent_itemsets:
        result.append((itemset, support))

    result = sorted(result, key=lambda i: i[0])
    for itemset, support in result:
        print(str(itemset) + ' ' + str(support/total_baskets))

    end = time.time()
    print('Execution time:', str(end - start))


['F11'] 0.2664113320566603
['F12'] 0.29085145425727127
['F13'] 0.28638143190715953
['F14'] 0.28571142855714277
['F15'] 0.2703363516817584
['F16'] 0.2653413267066335
['F17'] 0.2806864034320172
['F2'] 0.2963464817324087
['F20'] 0.26556132780663905
['F21'] 0.25038625193125963
['F22'] 0.29124145620728104
['F23'] 0.25608628043140214
['F24'] 0.2556162780813904
['F25'] 0.26524632623163114
['F27'] 0.29528147640738206
['F28'] 0.2714863574317872
['F3'] 0.28132140660703303
['F30'] 0.26656133280666405
['F31'] 0.3071515357576788
['F32'] 0.28114140570702856
['F33'] 0.27077135385676926
['F34'] 0.2510312551562758
['F35'] 0.2515512577562888
['F38'] 0.2555812779063895
['F39'] 0.2862964314821574
['F4'] 0.29654148270741354
['F40'] 0.28540642703213515
['F41'] 0.2763763818819094
['F42'] 0.31624658123290617
['F44'] 0.260601303006515
['F45'] 0.27573637868189343
['F47'] 0.2716363581817909
['F48'] 0.29642148210741054
['F5'] 0.2765563827819139
['F6'] 0.3109515547577738
['F7'] 0.25054125270626354
['F8'] 0.3160315

# eclat
https://github.com/jagdeep227/Frequent_itemset_mining/blob/main/Eclat_g.py


In [ ]:
# eclat
#eclat
import time

FreqItems = dict()
support = dict()


def eclat(prefix, items, dict_id ,minsup,cnt):
    
    while items:
        i ,itids = items.pop()
        isupp = len(itids)
        if isupp >= minsup*cnt:
            print(prefix + [i], isupp/len(dataset))
            FreqItems[frozenset(prefix + [i])] = isupp
            suffix = []
            for j, ojtids in items:
                jtids = itids & ojtids
                if len(jtids ) >= minsup*cnt:
                    suffix.append((j, jtids))
            dict_id += 1
            eclat(prefix +[i], sorted(suffix, key=lambda item: len(item[1]), reverse=True), dict_id ,minsup,cnt)

In [ ]:
items = {}

for transaction_id, transaction in enumerate(dataset):
    for item in transaction:
        if item in items:
            items[item].add(transaction_id)
        else:
            items[item] = {transaction_id}

    # Convert the dictionary items into a list of tuples
items = [(item, itids) for item, itids in items.items()]

In [ ]:
# Dataset
dataset = new_array
# Main function
if __name__ == '__main__':
    # Call find_frequent_itemsets() to generate frequent itemsets
    start = time.time()

    cnt = len(dataset)
    minsup = 0.25
    # Function call to start ECLAT algorithm
    eclat([], sorted(items, key=lambda item: len(item[1]), reverse=True), 1, minsup, cnt)

    result = []
    for itemset, support in frequent_itemsets:
        result.append((itemset, support))

    result = sorted(result, key=lambda i: i[0])
    for itemset, support in result:
        print(str(itemset) + ' ' + str(support/total_baskets))

    end = time.time()
    print('Execution time:', str(end - start))

['F21'] 0.25038625193125963
['F7'] 0.25054125270626354
['F34'] 0.2510312551562758
['F9'] 0.25106625533127663
['F35'] 0.2515512577562888
['F38'] 0.2555812779063895
['F24'] 0.2556162780813904
['F23'] 0.25608628043140214
['F44'] 0.260601303006515
['F25'] 0.26524632623163114
['F16'] 0.2653413267066335
['F20'] 0.26556132780663905
['F11'] 0.2664113320566603
['F30'] 0.26656133280666405
['F15'] 0.2703363516817584
['F33'] 0.27077135385676926
['F28'] 0.2714863574317872
['F47'] 0.2716363581817909
['F45'] 0.27573637868189343
['F41'] 0.2763763818819094
['F5'] 0.2765563827819139
['F17'] 0.2806864034320172
['F32'] 0.28114140570702856
['F3'] 0.28132140660703303
['F40'] 0.28540642703213515
['F14'] 0.28571142855714277
['F39'] 0.2862964314821574
['F13'] 0.28638143190715953
['F12'] 0.29085145425727127
['F22'] 0.29124145620728104
['F27'] 0.29528147640738206
['F2'] 0.2963464817324087
['F48'] 0.29642148210741054
['F4'] 0.29654148270741354
['F31'] 0.3071515357576788
['F6'] 0.3109515547577738
['F8'] 0.31603158